In [1]:
import tensorflow as tf 
import os
import pickle
import numpy as np

CIFAR_DIR = './cifar-10-batches-py/'
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    '''read data from the file'''
    with open(filename, 'rb') as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
#先到了sess.run（）发现数据传进来不是很合适，需要写数据处理方面的一些方法
'''可以去看Tensorflow.Dataset的API'''
class  CifarData: 
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_label = []
        for filename in filenames:
            data, labels = load_data(filename)
            '''少用一个for循环'''
            for item, label in zip(data,labels):
                if  label in [0,1]:
                    all_data.append(item)
                    all_label.append(label)
        #item是从data中来，data是矩阵，item就是向量，纵向合并起来把它做成一个矩阵
        self._data = np.vstack(all_data)
        '''做了归一化看训练集准确率是否有变化-----
        -----训练集和测试集明显提升且训练集准确率有明显变化'''
        self._data = self._data/127.5-1  
        '''因为值为01，结果在01之间，结果会偏行一方或者另一方而sigmoid函数在这种情况下会导致梯度消失'''
        self._labels = np.hstack(all_label)
    
        print(self._data.shape)
        print(self._labels.shape)
        self._num_example = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 #nimibutch塞了多少个，数据及遍历到哪个位置
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        #将0-self._num_example的数进行一个混合排列----数据及下标的混合排列
        p = np.random.permutation(self._num_example)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        '''返回batch_size个样本'''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_example:
            if self._need_shuffle :
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more example')
        if end_indicator > self._num_example:
            raise Exception("batch size is larger than all examples")
            
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator : end_indicator]
        self._indicator = end_indicator
        return batch_data,  batch_labels
        
train_filanames = [os.path.join(CIFAR_DIR,'data_batch_%d'%i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]


#测试
train_data = CifarData(train_filanames,True)
test_data =  CifarData(test_filenames,True)
# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)
'''----确认是可以正常工作的--------'''


(10000, 3072)
(10000,)
(2000, 3072)
(2000,)


'----确认是可以正常工作的--------'

In [3]:
#搭建计算图
x = tf.placeholder(tf.float32,[None,3072])  #输入的数量是不一定的
#[None]
y = tf.placeholder(tf.float64,[None])  #y随x变

#w跟每个样本做点积，维度应该是单个样本的维度3072，一个输出---【3072，1】
w = tf.get_variable('w',[x.get_shape()[-1],1],initializer=tf.random_normal_initializer(0,1))

#b跟w的第二维输出是一样的,通常使用常量来初始化----[1,]
b = tf.get_variable('b',[1],initializer=tf.constant_initializer(0.0))

#[None,3072] * [3072,1] = [None,1]
y_ = tf.matmul(x, w) + b
#[None,1]
p_y_1 = tf.nn.sigmoid(y_)  

#y的shape和p_y_1的shape不同啊，不能做计算
y_reshaped = tf.reshape(y,(-1,1))

#注意，tf对类型敏感，y-teshape是float64，p_y_1是float32，不行
y_reshaped = tf.cast(y_reshaped, tf.float32)


loss = tf.reduce_mean(tf.square(y_reshaped - p_y_1))

#bool
predict = p_y_1 > 0.5

#[1,0,1,1,0..........,1]
correct_predicttion = tf.equal(tf.cast(predict,tf.float32),y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_predicttion,tf.float64))

#梯度下降的方法
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [4]:
#要执行这个计算图要
init = tf.global_variables_initializer()
batch_size = 20
train_step = 100000
test_step =100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_step):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        
        loss_val,acc_val, _ = sess.run([loss,accuracy,train_op],
                                       feed_dict={x : batch_data,
                                                       y : batch_labels})
        if (i+1)%500 == 0 :
            print('train step : %d, loss : %4.5f, acc : %4.5f'%(i+1, loss_val, acc_val))
            
        if (i+1)%5000 == 0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_step):
                
                test_batch_data,test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict={
                        x : test_batch_data, 
                        y : test_batch_labels
                    })
                #记录所有test总的结果在做平均
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('test step: %d, acc:%4.5f'%(i+1,test_acc))
            '''结果发现测试集准确率 不变，说明没有学到什么东西
            为解决这个问题，对self._data等进行缩放'''

train step : 500, loss : 0.30000, acc : 0.70000
train step : 1000, loss : 0.14912, acc : 0.85000
train step : 1500, loss : 0.24300, acc : 0.75000
train step : 2000, loss : 0.15000, acc : 0.85000
train step : 2500, loss : 0.37887, acc : 0.60000
train step : 3000, loss : 0.20099, acc : 0.80000
train step : 3500, loss : 0.16677, acc : 0.80000


KeyboardInterrupt: 